In [1]:
import logging

In [2]:
import unittest

In [49]:
class Dispatcher:
    def __init__(self, n, v):
        self.n = n
        self.v = v
        self.cache_data = dict()
        self.new = list(range(n))
        self.processing = dict()
        
    def __repr__(self):
        return f"Job Dispatcher: n:{self.n},v{self.v}"
        
    def __getitem__(self, user_id):
        if user_id in self.cache_data:
            return self.cache_data[user_id]
        else:
            for k, v in self.processing.items():
                if len(self.processing[k]) >= self.v:
                    continue
                if user_id in v:
                    continue
                else:
                    v.append(user_id)
                    self.cache_data[user_id] = k
                    return k

        # read_new
        if len(self.new)>0:
            item = self.new[0]
            self.processing[item] = []
            self.new.remove(item)
            return self[user_id]
        else:
            return -1
    
    def finish_update(self, user_id, index):
        if user_id in self.cache_data:
            del self.cache_data[user_id]
        if index in self.processing:
            if len(self.processing[index]) >= self.v:
                del self.processing[index]
        

In [50]:
# from langhuan.progress import Dispatcher

In [51]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [52]:
class DispatcherTest(unittest.TestCase):
    def test_single_user(self,):
        dispatcher = Dispatcher(n=10, v=10)
        result = dict()
        for i in range(11):
            i1 = dispatcher[1]
            result.update({f"step_{i}": i1})
            dispatcher.finish_update(1, index=i1)
        expect = {'step_0': 0,
                  'step_1': 1,
                  'step_2': 2,
                  'step_3': 3,
                  'step_4': 4,
                  'step_5': 5,
                  'step_6': 6,
                  'step_7': 7,
                  'step_8': 8,
                  'step_9': 9,
                  'step_10': -1}
        self.assertEqual(result, expect)

    def test_nb_user_less_than_v(self):
        dispatcher = Dispatcher(n=10, v=10)
        result = dict()
        for i in range(7):
            i1, i2, i3 = dispatcher[1], dispatcher[2], dispatcher[3]
            result.update({f"step_{i}": [i1, i2, i3]})
            dispatcher.finish_update(1, index=i1)
            dispatcher.finish_update(3, index=i3)

        expected = {'step_0': [0, 0, 0],
                    'step_1': [1, 0, 1],
                    'step_2': [2, 0, 2],
                    'step_3': [3, 0, 3],
                    'step_4': [4, 0, 4],
                    'step_5': [5, 0, 5],
                    'step_6': [6, 0, 6]}
        self.assertEqual(result, expected)

    def test_nb_user_lg_than_v(self):
        dispatcher = Dispatcher(n=10, v=2)
        result = dict()
        for i in range(12):
            i1, i2, i3 = dispatcher[1], dispatcher[2], dispatcher[3]
#             logging.debug(f"{i1}, {i2}, {i3}")
            result.update({f"step_{i}": [i1, i2, i3]})
            if i % 3 != 0:
                dispatcher.finish_update(1, index=i1)
            if i % 3 != 1:
                dispatcher.finish_update(2, index=i2)
            if i % 3 != 2:
                dispatcher.finish_update(3, index=i3)
        print(result)
        expected = {
            'step_0': [0, 0, 1],
            'step_1': [0, 1, 2],
            'step_2': [2, 1, 3],
            'step_3': [3, 4, 3],
            'step_4': [3, 5, 4],
            'step_5': [5, 5, 6],
            'step_6': [6, 7, 6],
            'step_7': [6, 8, 7],
            'step_8': [8, 8, 9],
            'step_9': [9, -1, 9],
            'step_10': [9, -1, -1],
            'step_11': [-1, -1, -1]}

        self.assertEqual(result, expected)

In [53]:
ran = unittest.main(argv=['first-arg-is-ignored'], exit=False)

...

{'step_0': [0, 0, 1], 'step_1': [0, 1, 2], 'step_2': [2, 1, 3], 'step_3': [3, 4, 3], 'step_4': [3, 5, 4], 'step_5': [5, 5, 6], 'step_6': [6, 7, 6], 'step_7': [6, 8, 7], 'step_8': [8, 8, 9], 'step_9': [9, -1, 9], 'step_10': [9, -1, -1], 'step_11': [-1, -1, -1]}



----------------------------------------------------------------------
Ran 3 tests in 0.008s

OK
